# Coursera IBM Data Engineering: *Python Project for Data Engineering*
## Project & Scenario:
Create a code that can be used to:
1. **Extract** - compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD
2. **Transform** - the data needs to be transformed and stored in GBP, EUR and INR as well, in accordance with the exchange rate information that has been made available to you as a CSV file.
3. **Load** - the processed information table is to be saved locally in a CSV format and as a database table.

We need to use the following information to complete this project:

| Parameter	| Value |
------------|---------|
| Code name	| banks_project.py |
| Data URL	| https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks|
| Exchange rate CSV path | 	https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv|
| Table Attributes (upon Extraction only)	| Name, MC_USD_Billion|
| Table Attributes (final)	| Name, MC_USD_Billion, MC_GBP_Billion, MC_EUR_Billion, MC_INR_Billion|
| Output CSV Path	| ./Largest_banks_data.csv|
| Database name	| Banks.db|
| Table name	| Largest_banks|
| Log file	| code_log.txt|


### **Project tasks**

Task 1:
Write a function log_progress() to log the progress of the code at different stages in a file code_log.txt. Use the list of log points provided to create log entries as every stage of the code.

Task 2:
**Extract** the tabular information from the given URL under the heading 'By market capitalization' and save it to a dataframe.

1. *Inspect* the webpage and identify the position and pattern of the tabular information in the HTML code
2. *Write* the code for a function extract() to perform the required data extraction.
3. *Execute* a function call to extract() to verify the output.

Task 3:
**Transform** the dataframe by adding columns for Market Capitalization in GBP, EUR and INR, rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.

1. *Write* the code for a function transform() to perform the said task.
2. *Execute* a function call to transform() and verify the output.

Task 4:
**Load** the transformed dataframe to an output *CSV file*. Write a function load_to_csv(), execute a function call and verify the output.

Task 5:
**Load** the transformed dataframe to an *SQL database server* as a table. Write a function load_to_db(), execute a function call and verify the output.

Task 6:
Run queries on the database table. Write a function load_to_db(), execute a given set of queries and verify the output.

Task 7:
Verify that the log entries have been completed at all stages by checking the contents of the file code_log.txt.

In [67]:
# libraries imported for use
from bs4 import BeautifulSoup
import requests

import pandas as pd
import numpy as np

import csv
import sqlite3
from sqlalchemy import create_engine, MetaData, Table, Column, String, Float, text, select
from sqlalchemy.exc import OperationalError
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

## Task 1: Create Text File
I need to create a text file to save all of my data to while working.


In [68]:
# open new txt file
file = open("code_log.txt", "w")

# define function to write to file
def log_progress(string):
  file.write(string + '\n')


## Task 2: Extract
I need to extract information from both the URL and CSV file provided.

From the URL:
- I need to find the table that I need information from
- Then, I will store this information into a list to later make computations

From the CSV file:
- I need to simply put it into a dataframe

In [69]:
# @title
# extract and save CSV info into dataframe 1
def extract_csv(file):
    dataframe = pd.read_csv(file)
    return dataframe

df = extract_csv('/content/exchange_rate.csv')
print(df)

log_progress('CSV file extracted')

  Currency   Rate
0      EUR   0.93
1      GBP   0.80
2      INR  82.95


In [70]:
# @title
# extract from web page and save into dataframe 2
# df2 -> (which will be the one we will save and export as a CSV later)
# request url and format with html parser
url_mc = requests.get('https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks')
data_mc = BeautifulSoup(url_mc.content, 'html.parser')

#find table that I want
table = data_mc.find('table', {'class':'wikitable'})

# define headers
headers = [header.text.strip() for header in table.find_all('th')]

#create list
data = []

# loop through table and output results
rows = table.find_all('tr')
for row in rows[1:]:
    columns = row.find_all('td')
    if len(columns) > 0:
        row_data = [col.text.strip() for col in columns]
        data.append(row_data)

# write results in top_banks_df
top_banks_df = pd.DataFrame(data, columns=headers)

print(top_banks_df)

log_progress('Webpage extracted and added to top_banks_df')

  Rank                                Bank name Market cap(US$ billion)
0    1                           JPMorgan Chase                  432.92
1    2                          Bank of America                  231.52
2    3  Industrial and Commercial Bank of China                  194.56
3    4               Agricultural Bank of China                  160.68
4    5                                HDFC Bank                  157.91
5    6                              Wells Fargo                  155.87
6    7                        HSBC Holdings PLC                  148.90
7    8                           Morgan Stanley                  140.83
8    9                  China Construction Bank                  139.82
9   10                            Bank of China                  136.81


## Task 3: Transform
Now we need to transform the data:

1. They want us to use the column names "Name" and "MC_USD_Billion", so we will first change those before more analysis.

2. Then, they want us to use the exchange rate from df/csv file to show the Market cap in GBP, EUR, and INR currency rather than just USD. We will multiple the Market cap in top_banks_df by the rate in df/csv.

In [71]:
# update column names
top_banks_df.columns = ['Rank', 'Name', 'MC_USD_Billion']
print(top_banks_df)

log_progress('Updated column names and drop Rank column')

  Rank                                     Name MC_USD_Billion
0    1                           JPMorgan Chase         432.92
1    2                          Bank of America         231.52
2    3  Industrial and Commercial Bank of China         194.56
3    4               Agricultural Bank of China         160.68
4    5                                HDFC Bank         157.91
5    6                              Wells Fargo         155.87
6    7                        HSBC Holdings PLC         148.90
7    8                           Morgan Stanley         140.83
8    9                  China Construction Bank         139.82
9   10                            Bank of China         136.81


In [72]:
# transform and create columns
# make sure that all columns to calculate are numeric
top_banks_df['MC_USD_Billion'] = pd.to_numeric(top_banks_df['MC_USD_Billion'], errors='coerce')

# create variables for currency rates
EUR_rate = df.loc[df['Currency'] =='EUR', ['Rate']].values[0]
GBP_rate = df.loc[df['Currency'] =='GBP', ['Rate']].values[0]
INR_rate = df.loc[df['Currency'] =='INR', ['Rate']].values[0]

# define transform to calculate translate currency columns
def transform(column, currency_name):
    top_banks_df[f'MC_{currency_name}_Billion'] = np.round(top_banks_df['MC_USD_Billion'] * column, 2)

# pass currency rates
transform(EUR_rate, 'EUR')
transform(GBP_rate, 'GBP')
transform(INR_rate, 'INR')

# print new dataframe
print(top_banks_df)

log_progress('Transformed data in top_banks_df')


  Rank                                     Name  MC_USD_Billion  \
0    1                           JPMorgan Chase          432.92   
1    2                          Bank of America          231.52   
2    3  Industrial and Commercial Bank of China          194.56   
3    4               Agricultural Bank of China          160.68   
4    5                                HDFC Bank          157.91   
5    6                              Wells Fargo          155.87   
6    7                        HSBC Holdings PLC          148.90   
7    8                           Morgan Stanley          140.83   
8    9                  China Construction Bank          139.82   
9   10                            Bank of China          136.81   

   MC_EUR_Billion  MC_GBP_Billion  MC_INR_Billion  
0          402.62          346.34        35910.71  
1          215.31          185.22        19204.58  
2          180.94          155.65        16138.75  
3          149.43          128.54        13328.41  
4 

## Task 4: Export to CSV file.

In [73]:
def load_to_csv(csvfile):
    top_banks_df.to_csv(csvfile, index=False)

load_to_csv('./Largest_banks_data.csv');
new_df = pd.read_csv('./Largest_banks_data.csv')
print(new_df)

log_progress('CSV file exported using top_banks_df')

   Rank                                     Name  MC_USD_Billion  \
0     1                           JPMorgan Chase          432.92   
1     2                          Bank of America          231.52   
2     3  Industrial and Commercial Bank of China          194.56   
3     4               Agricultural Bank of China          160.68   
4     5                                HDFC Bank          157.91   
5     6                              Wells Fargo          155.87   
6     7                        HSBC Holdings PLC          148.90   
7     8                           Morgan Stanley          140.83   
8     9                  China Construction Bank          139.82   
9    10                            Bank of China          136.81   

   MC_EUR_Billion  MC_GBP_Billion  MC_INR_Billion  
0          402.62          346.34        35910.71  
1          215.31          185.22        19204.58  
2          180.94          155.65        16138.75  
3          149.43          128.54        13

## Task 5/6: Load to Database & Run Queries

In [74]:
# connect to server and load new CSV file
def load_to_db(dataframe):
    # create database and assign metadata
    engine = create_engine('sqlite:///Banks.db')
    metadata = MetaData()

    # create table and schema
    largest_banks = Table(
        'Largest_banks',
        metadata,
        Column('Name', String, primary_key=True),
        Column('MC_USD_Billion', Float),
        Column('MC_GBP_Billion', Float),
        Column('MC_EUR_Billion', Float),
        Column('MC_INR_Billion', Float)
    )

    # Execute queries
    with engine.connect() as connection:
        # Convert the DataFrame to SQL
        dataframe.to_sql('Largest_banks', con=engine, if_exists='replace', index=False)

        # Query the database
        print("Fetching all records:")
        stmt = select(largest_banks)
        result = connection.execute(stmt)
        for row in result:
            print(row)

        # select banks in table with only GBP Market Capilatization conversion
        london_query = text("SELECT Name, MC_GBP_Billion FROM Largest_banks")
        result = connection.execute(london_query)
        print("\n")
        print(result.fetchall())

        # select banks in table with only EUR Market Capilatization conversion
        berlin_query = text("SELECT Name, MC_EUR_Billion FROM Largest_banks")
        result = connection.execute(berlin_query)
        print("\n")
        print(result.fetchall())

        # select banks in table with only INR Market Capilatization conversion
        delhi_query = text("SELECT Name, MC_INR_Billion FROM Largest_banks")
        result = connection.execute(delhi_query)
        print("\n")
        print(result.fetchall())

# run new_df through function
load_to_db(new_df)

log_progress('Database loaded and queries executed')

Fetching all records:
('JPMorgan Chase', 432.92, 346.34, 402.62, 35910.71)
('Bank of America', 231.52, 185.22, 215.31, 19204.58)
('Industrial and Commercial Bank of China', 194.56, 155.65, 180.94, 16138.75)
('Agricultural Bank of China', 160.68, 128.54, 149.43, 13328.41)
('HDFC Bank', 157.91, 126.33, 146.86, 13098.63)
('Wells Fargo', 155.87, 124.7, 144.96, 12929.42)
('HSBC Holdings PLC', 148.9, 119.12, 138.48, 12351.26)
('Morgan Stanley', 140.83, 112.66, 130.97, 11681.85)
('China Construction Bank', 139.82, 111.86, 130.03, 11598.07)
('Bank of China', 136.81, 109.45, 127.23, 11348.39)


[('JPMorgan Chase', 346.34), ('Bank of America', 185.22), ('Industrial and Commercial Bank of China', 155.65), ('Agricultural Bank of China', 128.54), ('HDFC Bank', 126.33), ('Wells Fargo', 124.7), ('HSBC Holdings PLC', 119.12), ('Morgan Stanley', 112.66), ('China Construction Bank', 111.86), ('Bank of China', 109.45)]


[('JPMorgan Chase', 402.62), ('Bank of America', 215.31), ('Industrial and Commercia

## Task 7: Close File and Check Log

In [75]:
# close file and print file to double check code progress was loggeds
file.close()

with open("./code_log.txt", "r") as file:
    content = file.read()
    print(content)


CSV file extracted
Webpage extracted and added to top_banks_df
Updated column names and drop Rank column
Transformed data in top_banks_df
CSV file exported using top_banks_df
Database loaded and queries executed



# Project Credit/Info:

*Student/Author*: Bailey Davidson

January 3rd, 2025